In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import text,sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from sklearn.model_selection import train_test_split
print(tf.__version__)

2.2.0


In [22]:
train_df = pd.read_csv('BBC News Train.csv').fillna(' ')


In [3]:
x = train_df['Text'].values

In [4]:
labels = {'tech': 1, 'business':2, 'politics': 3 ,'entertainment': 4, 'sport' : 5}
y = train_df['Category'].values
for i in range(len(y)):
    y[i] = labels[y[i]]

In [5]:
print(train_df.sample(10))

      ArticleId                                               Text Category
425        1159  disney settles disclosure charges walt disney ...        2
1069        290  sydney to host north v south game sydney will ...        5
790         940  dame julie pops in to see poppins mary poppins...        4
928        1710  assembly ballot papers  missing  hundreds of b...        3
1304        888  minister defends hunting ban law the law banni...        3
72         1766  vickery upbeat about arm injury england prop p...        5
1078       1963  south african car demand surges car manufactur...        2
1163        909  british library gets wireless net visitors to ...        1
37          805  yachvili savours france comeback france scrum-...        5
1467        937  charvis set to lose fitness bid flanker colin ...        5


In [6]:
max_features = 20000
max_text_length = 100

In [7]:
x_tokenizer = text.Tokenizer(max_features)
x_tokenizer.fit_on_texts(list(x))
x_tokenized = x_tokenizer.texts_to_sequences(x)
x_train_val = sequence.pad_sequences(x_tokenized,maxlen=max_text_length)

In [9]:
embedding_dim = 100
embeddings_index = dict()
f = open('glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype ='float32')
    embeddings_index[word] = coefs
f.close()
print(f'Found {len(embeddings_index)} word vectors')

Found 14673 word vectors


In [10]:
embedding_matrix = np.zeros((max_features,embedding_dim))
for word,index in x_tokenizer.word_index.items():
    if (index>max_features-1):
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [11]:
from keras.models import *
from keras.layers import *
from keras.callbacks import *

model = Sequential()
'''model.add(Embedding(max_features,
                   embedding_dim,
                   embeddings_initializer = tf.keras.initializers.Constant(
                   embedding_matrix),
                   trainable = False))'''

model.add(Embedding(max_features,100,input_length=100,trainable=True)) 

#lstm layer
model.add(LSTM(128,return_sequences=True,dropout=0.2))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5,activation ='softmax'))
model.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          2000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 128)          117248    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
Total params: 2,125,829
Trainable params: 2,125,829
Non-trainable params: 0
____________________________________________

In [12]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = 'adam',
              metrics =['accuracy'])

In [13]:
x_train, x_val, y_train, y_val=  train_test_split(x_train_val, y,
                                                  test_size =0.10, random_state = 1)


In [14]:
batch_size = 32

model.fit(x_train, y_train,
         batch_size = batch_size,
         epochs = 10,
         validation_data = (x_val,y_val))


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1341 samples, validate on 149 samples
Epoch 1/10
1341/1341 [==============================] - 14s 11ms/step - loss: nan - accuracy: 0.0037 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/10
1341/1341 [==============================] - 12s 9ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/10
1341/1341 [==============================] - 12s 9ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/10
1341/1341 [==============================] - 13s 9ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/10
1341/1341 [==============================] - 12s 9ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 6/10
1341/1341 [==============================] - 12s 9ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 7/10
1341/1341 [==============================] - 13s 9ms/step - los

In [15]:
test_df = pd.read_csv('BBC News Train.csv')
x_test = test_df['Text'].values


In [19]:
x_test_tokenized = x_tokenizer.texts_to_sequences(x_test)
x_testing = sequence.pad_sequences(x_test_tokenized,maxlen = 100)

In [20]:
y_testing = model.predict(x_testing, verbose=1,batch_size=32)

1490/1490 [==============================] - 1s 813us/step
